In [1]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from binascii import hexlify, unhexlify


def hex_to_bits(hex_string):
    # Convert hex to binary
    binary_representation = bin(int(hex_string, 16))[2:]
    
    # Ensure leading zeros are included
    length = len(hex_string) * 4  # Each hex digit represents 4 bits
    return binary_representation.zfill(length)

def string_to_bits(input_string):
    return ''.join(format(ord(char), '08b') for char in input_string)


def bits_to_string(bits):
    return ''.join(chr(int(bits[i:i+8], 2)) for i in range(0, len(bits), 8))


def encrypt_string(input_string, key):
    # Convert string to bytes
    input_bytes = input_string.encode('utf-8')

    # Pad the data
    padded_data = pad(input_bytes, AES.block_size)

    # Generate a random IV
    iv = get_random_bytes(AES.block_size)

    # Create AES cipher object
    cipher = AES.new(key, AES.MODE_CBC, iv)

    # Encrypt the data
    ciphertext = cipher.encrypt(padded_data)

    # Convert the ciphertext to bit representation
    ciphertext_bits = hex_to_bits(hexlify(ciphertext).decode('utf-8'))

    return iv, ciphertext_bits

# Example usage:
key = get_random_bytes(AES.block_size)  # Generate a random key
input_string = "123456789abcdef"

# Step 1: Convert string to bits
input_bits = string_to_bits(input_string)
print("Input String (in bits):", input_bits)

# Step 2: Encrypt the string
iv, ciphertext_bits = encrypt_string(input_string, key)
print("Ciphertext (in bits):", ciphertext_bits)

# Step 3: Decrypt the ciphertext
# decrypted_string = decrypt_bits(ciphertext_bits, key, iv)
# print("Decrypted String:", decrypted_string)

Input String (in bits): 001100010011001000110011001101000011010100110110001101110011100000111001011000010110001001100011011001000110010101100110
Ciphertext (in bits): 01010101101101100010110100010100101011101110111110100110010100110111111001011111010101111011100001010101011010010110111011111010


In [2]:
len(input_bits),len(ciphertext_bits)

(120, 128)

In [3]:
import nltk
from nltk.corpus import genesis

In [4]:
genesis_words = genesis.words()

In [5]:
def filter_letter(let):

    if ord('a') <= ord(let) <= ord('z'):
        return True

    else:
        return False

In [6]:
def make_blocks(word_list, block_size):
    word_str = "".join(word_list)
    new_str = ""
    blocked_arr = []

    for i in range(0, len(word_str)):

        let = word_str[i].lower()
        if filter_letter(let):
            new_str += let
        else:
            pass

    del word_str

    for i in range(0, len(new_str), block_size):
        block = new_str[i:i+block_size]

        if len(block) == block_size:
            blocked_arr.append(block)
        else:
            diff = block_size - len(block)
            padded_block = block + "x" * diff
            blocked_arr.append(padded_block)

    return blocked_arr

In [7]:
pt_blocks = make_blocks(genesis_words, 8)

In [8]:
first_block = pt_blocks[0]

In [9]:
len(string_to_bits(first_block))

64

In [10]:
_,ct_bits = encrypt_string(first_block,key)
len(ct_bits)

128

In [11]:
len(hex_to_bits(hexlify(key)))

128

In [12]:
pt_bits = []
for pt in pt_blocks:
    pt_bits.append(string_to_bits(pt))


In [13]:
ct_bits = []

for pt in pt_blocks:
    # ct is in bits
    _,ct = encrypt_string(pt,key)
    ct_bits.append(ct)


# Converting to a dataset

In [14]:
import numpy as np

In [15]:
for bits in pt_bits:
    if bits.isnumeric():
        pass
    else:
        print("Don't Panic")
for bits in ct_bits:
    if bits.isnumeric():
        pass
    else:
        print("Don't Panic")

In [16]:
ptb_array = []
ctb_array = []

# 3 is ending token, 2 is starting token
for ptb in pt_bits:
    x = [int(i) for i in list(ptb)] + [3]
    ptb_array.append(x)

for ctb in ct_bits:
    x = [2] + [int(i) for i in list(ctb)] + [3]
    ctb_array.append(x)


    

In [17]:
ptb_array = np.array(ptb_array)
ctb_array = np.array(ctb_array)
# for a fixed key

In [18]:
ptb_array.shape,ctb_array.shape

((134339, 65), (134339, 130))

# Making the model

In [19]:
inbits_array = ptb_array
outbits_arrray = ctb_array

In [20]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [21]:
unique_chars = set()

for word in ptb_array:

    for let in word:

        if not let in unique_chars:
            unique_chars.add(let)


print(len(unique_chars))

3


In [22]:
batch_size = 64
num_samples = 134339
latent_dim = 256
epochs = 10

In [23]:
num_encoder_tokens = 4 # 0,1,3,' '
num_decoder_tokens = 5 # 0,1,2,3,' '


max_encoder_seq_length = max([len(pt) for pt in inbits_array])
max_decoder_seq_length = max([len(pt) for pt in outbits_arrray])


print("Number of samples:", len(inbits_array))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 134339
Number of unique input tokens: 4
Number of unique output tokens: 5
Max sequence length for inputs: 65
Max sequence length for outputs: 130


In [24]:
input_token_index = {0:0,1:1,3:2,' ':3}
target_token_index = {0: 0, 1: 1, 2:2, 3: 3, ' ': 4}


encoder_input_data = np.zeros(
    (len(inbits_array), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)


decoder_input_data = np.zeros(


    (len(inbits_array), max_decoder_seq_length, num_decoder_tokens),


    dtype="float32",
)


decoder_target_data = np.zeros(


    (len(inbits_array), max_decoder_seq_length, num_decoder_tokens),


    dtype="float32",
)

In [25]:
encoder_input_data.shape

(134339, 65, 4)

In [26]:
for i, (input_text, target_text) in enumerate(zip(inbits_array,outbits_arrray)):
    for t, char in enumerate(input_text):
        
        encoder_input_data[i, t, input_token_index[char]] = 1.0

    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [27]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)


encoder_states = [state_h, state_c]


decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(
    latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(
    decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
epochs = 3
model.compile(
    optimizer="rmsprop"
    , loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_aes_model.keras")

Epoch 1/3
 931/1680 [===============>..............] - ETA: 2:34 - loss: 0.6897 - accuracy: 0.5059

KeyboardInterrupt: 